In [1]:
!pip install Owlready2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.4/27.4 MB 33.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for Owlready2: filename=Owlready2-0.41-cp39-cp39-linux_x86_64.whl size=24160920 sha256=5dfae5796681bfae3cfd1d0c6156d77933a427d95c6f4cfba737084a59b59111
  Stored in directory: /root/.cache/pip/wheels/c4/56/67/10bd2f89fc6262317ee02f4d8dbdff95f1db434b6bb18daed0
Successfully built Owlready2


In [9]:
!pip install ngram

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [13]:
!pip install py-stringmatching

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 643.8/643.8 kB 8.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for py-stringmatching: filename=py_stringmatching-0.4.3-cp39-cp39-linux_x86_64.whl size=2601563 sha256=4dd0a7df2ff2785c84641e6bbb5f62a27b4bc4100b6bd461dd459e75a3cff928
  Stored in directory: /root/.cache/pip/wheels/24/f9/e2/89e3ea9801245b19c9c6c365eb4c25afde526cea5e1e296ad9
Successfully built py-stringmatching


In [22]:
import itertools
from re import finditer
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from owlready2 import get_ontology
from sklearn.metrics import f1_score

In [32]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [26]:
import ngram
from gensim.models import KeyedVectors
from nltk.corpus import wordnet
from py_stringmatching.similarity_measure.affine import Affine
from py_stringmatching.similarity_measure.bag_distance import BagDistance
from py_stringmatching.similarity_measure.cosine import Cosine
from py_stringmatching.similarity_measure.dice import Dice
from py_stringmatching.similarity_measure.editex import Editex
from py_stringmatching.similarity_measure.generalized_jaccard import GeneralizedJaccard
from py_stringmatching.similarity_measure.jaccard import Jaccard
from py_stringmatching.similarity_measure.jaro import Jaro
from py_stringmatching.similarity_measure.jaro_winkler import JaroWinkler
from py_stringmatching.similarity_measure.levenshtein import Levenshtein
from py_stringmatching.similarity_measure.monge_elkan import MongeElkan
from py_stringmatching.similarity_measure.needleman_wunsch import NeedlemanWunsch
from py_stringmatching.similarity_measure.overlap_coefficient import OverlapCoefficient
from py_stringmatching.similarity_measure.partial_ratio import PartialRatio
from py_stringmatching.similarity_measure.partial_token_sort import PartialTokenSort
from py_stringmatching.similarity_measure.ratio import Ratio
from py_stringmatching.similarity_measure.smith_waterman import SmithWaterman
from py_stringmatching.similarity_measure.soft_tfidf import SoftTfIdf
from py_stringmatching.similarity_measure.soundex import Soundex
from py_stringmatching.similarity_measure.tfidf import TfIdf
from py_stringmatching.similarity_measure.token_sort import TokenSort
from py_stringmatching.similarity_measure.tversky_index import TverskyIndex
from tqdm import tqdm

In [18]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Utils for Datasets

In [3]:
def read_ontology(path):
    onto = get_ontology(path)
    onto.load()

    # read classes
    classes = []
    for cls in onto.classes():
        classes.append(cls)
    # print(classes)
    # print(len(classes))
    # print(len(set(classes)))
    classes = list(set(classes))  # removing any number of repetitions of a class

    # read properties
    properties = []
    for prop in onto.properties():
        properties.append(prop)
    properties = list(set(properties))

    return classes, properties

In [4]:
def get_mappings(path):
    mappings = []
    with open(path) as map_file:
        soup = BeautifulSoup(map_file, 'xml')

    for cell in soup.find_all('Cell'):
        entity1 = cell.find('entity1')['rdf:resource'].split('#')[1]
        entity2 = cell.find('entity2')['rdf:resource'].split('#')[1]
        mapping = (entity1, entity2)
        mappings.append(mapping)

    return mappings

In [5]:
def get_path(cls):
    path = cls.name

    while True:
        try:
            path = path + '/' + cls.is_a[0].name
        except:
            break
        cls = cls.is_a[0]
        if cls == 'owl.Thing':
            break
    
    return '/'.join(path.split('/')[::-1])

In [6]:
def get_dataset(onto1, onto2, map_file):
    data = []

    mappings = get_mappings(map_file)
    classes1, properties1 = read_ontology(onto1)
    classes2, properties2 = read_ontology(onto2)

    class_pairs = list(itertools.product(classes1, classes2))
    for pair in class_pairs:
        temp = pair
        pair = (pair[0].name, pair[1].name)
        if pair in mappings:
            matched = 1
        else:
            matched = 0

        data.append((onto1, onto2, pair[0], pair[1], temp[0], temp[1], get_path(temp[0]), get_path(temp[1]), matched, 'Class'))

    property_pairs = list(itertools.product(properties1, properties2))
    for pair in property_pairs:
        temp = pair
        pair = (pair[0].name, pair[1].name)
        if pair in mappings:
            matched = 1
        else:
            matched = 0

        data.append((onto1, onto2, pair[0], pair[1], temp[0], temp[1], get_path(temp[0]), get_path(temp[1]), matched, 'Property'))

    df = pd.DataFrame(data, columns=['Ontology1', 'Ontology2', 'Entity1', 'Entity2', 'Parent1', 'Parent2', 'Full Path1', 'Full Path2', 'Match', 'Type'])
    return df

# Utils for Features

In [19]:
!unzip /content/drive/MyDrive/GoogleNews-vectors-negative300.bin.zip

Archive:  /content/drive/MyDrive/GoogleNews-vectors-negative300.bin.zip
  inflating: GoogleNews-vectors-negative300.bin  


In [23]:
def camel_case_split(identifier):
    matches = finditer('.+?(?:(?<=[a-z])(?=[A-Z])|(?<=[A-Z])(?=[A-Z][a-z])|$)',
                       identifier)
    return [m.group(0) for m in matches]

In [24]:
def get_word2vec_sim(row_set1, row_set2):
    sum_sim = 0
    N = max(len(row_set1), len(row_set2))

    for w1 in row_set1:
        maxSim = 0
        for w2 in row_set2:
            try:
                sim = model.wv.similarity(w1, w2)
            except:
                sim = 0

            if sim > maxSim:
                maxSim = sim
        sum_sim = sum_sim + maxSim

    sum_sim = sum_sim / N

    return sum_sim

In [25]:
def get_words(text):
    if '_' in text:
        row_set = text.split('_')
    else:
        if '-' in text:
            row_set = text.split('-')
        else:
            row_set = camel_case_split(text)

    row_set = [x.lower() for x in row_set]
    return row_set

In [27]:
def calculate_features(dataset, string_type):
    ngrams1 = []
    ngrams2 = []
    ngrams3 = []
    ngrams4 = []
    dices = []
    jaccards = []
    jaros = []
    mes = []
    sws = []
    afs = []
    bds = []
    coses = []
    prs = []
    sfs = []
    edxs = []
    gjs = []
    jws = []
    lws = []
    ptss = []
    rats = []
    sounds = []
    tfidfs = []
    tss = []
    tvs = []
    ovs = []
    nws = []
    wordnet_sims = []
    w2vec_sims = []

    if string_type == 'Entity':
        index = 2
    elif string_type == 'Parent':
        index = 4
    elif string_type == 'Path':
        index = 6

    for key, row in tqdm(dataset.iterrows()):

        string1 = row[index]
        string2 = row[index + 1]

        ngrams1.append(ngram.NGram.compare(string1, string2, N=1))
        ngrams2.append(ngram.NGram.compare(string1, string2, N=2))
        ngrams3.append(ngram.NGram.compare(string1, string2, N=3))
        ngrams4.append(ngram.NGram.compare(string1, string2, N=4))
        lws.append(lev.get_sim_score(string1, string2))
        jaros.append(jaro.get_sim_score(string1, string2))
        nws.append(nw.get_raw_score(string1, string2))
        sws.append(sw.get_raw_score(string1, string2))
        afs.append(af.get_raw_score(string1, string2))
        bds.append(bd.get_sim_score(string1, string2))
        prs.append(pr.get_sim_score(string1, string2))
        edxs.append(edx.get_sim_score(string1, string2))
        ptss.append(pts.get_sim_score(string1, string2))
        rats.append(rat.get_sim_score(string1, string2))
        sounds.append(sound.get_sim_score(string1, string2))
        tss.append(ts.get_sim_score(string1, string2))
        jws.append(jw.get_sim_score(string1, string2))

        row_set1 = get_words(string1)
        row_set2 = get_words(string2)

        mes.append(me.get_raw_score(row_set1, row_set2))
        coses.append(cos.get_sim_score(row_set1, row_set2))
        sfs.append(sf.get_raw_score(row_set1, row_set2))
        gjs.append(gj.get_sim_score(row_set1, row_set2))
        tfidfs.append(tfidf.get_sim_score(row_set1, row_set2))
        tvs.append(tv_ind.get_sim_score(row_set1, row_set2))
        ovs.append(over_coef.get_sim_score(row_set1, row_set2))
        dices.append(dice.get_sim_score(row_set1, row_set2))
        jaccards.append(jac.get_sim_score(row_set1, row_set2))

        allsyns1 = set(ss for word in row_set1 for ss in wordnet.synsets(word))
        allsyns2 = set(ss for word in row_set2 for ss in wordnet.synsets(word))

        best = [wordnet.wup_similarity(s1, s2) for s1, s2 in itertools.product(allsyns1, allsyns2)]
        if len(best) > 0:
            wordnet_sims.append(best[0])
        else:
            wordnet_sims.append(0)

        w2vec_sims.append(get_word2vec_sim(row_set1, row_set2))

    dataset['Ngram1' + '_' + string_type] = ngrams1
    dataset['Ngram2' + '_' + string_type] = ngrams2
    dataset['Ngram3' + '_' + string_type] = ngrams3
    dataset['Ngram4' + '_' + string_type] = ngrams4
    dataset['Dice' + '_' + string_type] = dices
    dataset['Jaccard' + '_' + string_type] = jaccards
    dataset['Jaro' + '_' + string_type] = jaros
    dataset['Monge-Elkan' + '_' + string_type] = mes
    dataset['SmithWaterman' + '_' + string_type] = sws
    dataset['AffineGap' + '_' + string_type] = afs
    dataset['BagDistance' + '_' + string_type] = bds
    dataset['Cosine_similarity' + '_' + string_type] = coses
    dataset['PartialRatio' + '_' + string_type] = prs
    dataset['Soft_TFIDF' + '_' + string_type] = sfs
    dataset['Editex' + '_' + string_type] = edxs
    dataset['GeneralizedJaccard' + '_' + string_type] = gjs
    dataset['JaroWinkler' + '_' + string_type] = jws
    dataset['Levenshtein' + '_' + string_type] = lws
    dataset['PartialTokenSort' + '_' + string_type] = ptss
    dataset['Ratio' + '_' + string_type] = rats
    dataset['Soundex' + '_' + string_type] = sounds
    dataset['TFIDF' + '_' + string_type] = tfidfs
    dataset['TokenSort' + '_' + string_type] = tss
    dataset['TverskyIndex' + '_' + string_type] = tvs
    dataset['OverlapCoef' + '_' + string_type] = ovs
    dataset['Needleman-Wunsch' + '_' + string_type] = nws
    dataset['Wordnet_sim' + '_' + string_type] = wordnet_sims
    dataset['Word2vec_sim' + '_' + string_type] = w2vec_sims

    return dataset

In [28]:
af = Affine()
me = MongeElkan()
nw = NeedlemanWunsch()
sw = SmithWaterman()
bd = BagDistance()
cos = Cosine()
pr = PartialRatio()
sf = SoftTfIdf()
edx = Editex()
gj = GeneralizedJaccard()
jw = JaroWinkler()
lev = Levenshtein()
dice = Dice()
jac = Jaccard()
jaro = Jaro()
pts = PartialTokenSort()
rat = Ratio()
sound = Soundex()
tfidf = TfIdf()
ts = TokenSort()
tv_ind = TverskyIndex()
over_coef = OverlapCoefficient()
model = KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)

# Create Dataset

In [30]:
ontologies = ['101.rdf', '102.rdf', '103.rdf', '301.rdf', '302.rdf', '303.rdf', '304.rdf']
train_aligns = ['101-102.rdf', '101-103.rdf', '101-301.rdf']
test_aligns = ['101-302.rdf', '101-303.rdf', '101-304.rdf']

train_datasets = []

# Create dataset
for align in train_aligns:
    ont1, ont2 = align.split('.')[0].split('-')

    ont1_path = '/content/' + ont1 + '.rdf'
    ont2_path = '/content/' + ont2 + '.rdf'
    alignment_path = '/content/' + align

    train_datasets.append(get_dataset(ont1_path, ont2_path, alignment_path))

train = pd.concat(train_datasets, ignore_index=True)

test_datasets = []

for align in test_aligns:
    ont1, ont2 = align.split('.')[0].split('-')

    ont1_path = '/content/' + ont1 + '.rdf'
    ont2_path = '/content/' + ont2 + '.rdf'
    alignment_path = '/content/' + align

    test_datasets.append(get_dataset(ont1_path, ont2_path, alignment_path))

test = pd.concat(test_datasets, ignore_index=True)

train.to_csv('/content/dataset_train.csv', index=False)
test.to_csv('/content/dataset_test.csv', index=False)

# Calculate Features

In [33]:
# Calculate features for training dataset
data = pd.read_csv('dataset_train.csv')

data = calculate_features(data, 'Entity')
data = calculate_features(data, 'Parent')
data = calculate_features(data, 'Path')

data.to_csv('dataset_train_features.csv', index=False)

# Calculate features for testing dataset
data = pd.read_csv('dataset_test.csv')

data = calculate_features(data, 'Entity')
data = calculate_features(data, 'Parent')
data = calculate_features(data, 'Path')

data.to_csv('dataset_test_features.csv', index=False)

14148it [04:41, 50.34it/s]
14148it [00:59, 236.62it/s]
14148it [01:59, 118.42it/s]
14940it [04:16, 58.33it/s]
14940it [01:03, 236.59it/s]
14940it [01:59, 124.73it/s]


# Train and Evaluate

In [61]:
from sklearn.linear_model import LogisticRegression

def train_lr_model(X_train, y_train, X_test, y_test):
    model = LogisticRegression(penalty='l2', C=7.742637, class_weight=None)

    # Train model
    model.fit(X_train, y_train)
    y_prob = model.predict_proba(X_test)

    return y_prob

In [65]:
from sklearn.ensemble import RandomForestClassifier

def train_rf_model(X_train, y_train, X_test,y_test):
    model = RandomForestClassifier(n_estimators=400, max_features='sqrt', max_depth=5, random_state=42)

    # Train model
    model.fit(X_train, y_train)
    y_prob = model.predict_proba(X_test)

    return y_prob

In [63]:
import xgboost as xgb

def train_xgb_model(Xy_train, Xy_test):
    param = {'silent': 0, 'objective': 'binary:logistic',
                'min_child_weight': 10, 'gamma': 2.0, 'subsample': 0.8,
                'colsample_bytree': 0.8, 'max_depth': 10, 'nthread': 6,
                'eval_metric': 'error'}
    evallist = [(Xy_test, 'eval'), (Xy_train, 'train')]
    num_round = 20
    
    # Train model
    bst = xgb.train(param, Xy_train, num_round, evallist,
                    verbose_eval=False)

    y_prob = bst.predict(Xy_test)

    return y_prob

In [68]:
selected_model = 'RandomForest'

train_features = pd.read_csv('dataset_train_features.csv')
test_features = pd.read_csv('dataset_test_features.csv')

# Create feature "Type" for training dataset
train_types = []

for row in train_features['Type']:
    if row == 'Class':
        train_types.append(1)
    else:
        train_types.append(0)

train_features['Type_encode'] = train_types

# Create feature "Type" for testing dataset
test_types = []

for row in test_features['Type']:
    if row == 'Class':
        test_types.append(1)
    else:
        test_types.append(0)

test_features['Type_encode'] = test_types

X_train = train_features.loc[:, 'Ngram1_Entity':'Type_encode']
y_train = train_features['Match']

X_test = test_features.loc[:, 'Ngram1_Entity':'Type_encode']
y_test = test_features['Match']

df_train = train_features.loc[:, 'Ngram1_Entity':'Type_encode']
df_train['Match'] = train_features['Match']

df_test = test_features.loc[:, 'Ngram1_Entity':'Type_encode']
df_test['Match'] = test_features['Match']

# Fill nan values with zero
X_train = X_train.fillna(value=0)
X_test = X_test.fillna(value=0)

train = pd.read_csv('dataset_train.csv')
test = pd.read_csv('dataset_test.csv')

if selected_model != 'XGBoost':
    if selected_model == 'LogisticRegression':
        y_prob = train_lr_model(X_train, y_train, X_test, y_test)
    elif selected_model == 'RandomForest':
        y_prob = train_rf_model(X_train, y_train, X_test, y_test)
else:
    dtrain = xgb.DMatrix(X_train, label=y_train)
    dtest = xgb.DMatrix(X_test, label=y_test)
    y_prob = train_xgb_model(dtrain, dtest)

# Choose best threshold
for align in test_aligns:
    ont1, ont2 = align.split('.')[0].split('-')
    best_ts = 0
    best_score = 0

    for ts in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]:

        preds = []

        if selected_model != 'XGBoost':
            for x in y_prob:
                if x[1] >= ts:
                    preds.append(1)
                else:
                    preds.append(0)
        else:
            for x in y_prob:
                if x >= ts:
                    preds.append(1)
                else:
                    preds.append(0)

        test['Predict'] = preds

        pred_mappings = test[(test['Ontology1'] == f"/content/{ont1}.rdf") & (test['Ontology2'] == f"/content/{ont2}.rdf") & (test['Predict'] == 1)]

        true_mappings = test[(test['Ontology1'] == f"/content/{ont1}.rdf") & (test['Ontology2'] == f"/content/{ont2}.rdf") & (test['Match'] == 1)]

        correct_mappings = test[(test['Ontology1'] == f"/content/{ont1}.rdf") & (test['Ontology2'] == f"/content/{ont2}.rdf") & (test['Match'] == 1) & (test['Predict'] == 1)]

        true_count = len(true_mappings)
        pred_count = len(pred_mappings)
        corr_count = len(correct_mappings)

        if pred_count != 0 and true_count != 0 and corr_count != 0:
            precision = corr_count / pred_count
            recall = corr_count / true_count
            score = 2 * precision * recall / (precision + recall)
        else:
            score = 0

        if score > best_score:
            best_score = score
            best_ts = ts
            best_preds = preds

    print(
        f"Best F1-Score for {align} is {best_score} with threshold {best_ts}")

Best F1-Score for 101-302.rdf is 0.7012987012987012 with threshold 0.3
Best F1-Score for 101-303.rdf is 0.7959183673469388 with threshold 0.3
Best F1-Score for 101-304.rdf is 0.9240506329113924 with threshold 0.4
